# Draft Assistant

Run the cell below to use an interactive draft assistant.

In [ ]:
import statisticaldrafting as sd

sd.DraftAssistant(set="FDN", picks_shown=10).draft()

Button(button_style='warning', description='New Draft', style=ButtonStyle())

 Card Name                 | Rarity    | Color        | Synergy| Rating



Collection:
